# Richards equation

In this tutorial we present how to solve a Richards equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,2)\times(0,3)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$K$ the matrix conductivity, we want to solve the following problem: find $({q}, \psi)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary and initial conditions:

$$
\hat{\psi}(t,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
-2+2.2 \frac{t}{\Delta t_D}, \text{ on } \Gamma_{D_1}, t\leq\Delta t_D\\
0.2, \text{ on } \Gamma_{D_1}, t>\Delta t_D\\
1-z, \text{ on } \Gamma_{D_2}
\end{array}
\end{array}
\right., \qquad \nu \cdot q = 0 \text{ on } \Gamma_N \qquad \psi(0,x,z) = 1-z \text{ on } \Omega$$
where
$$
\Gamma_{D_1} = \left\{ (x,z) \in \partial \Omega \:|\: x \in [0,1] \wedge z=3  \right\},\\
\Gamma_{D_2} = \left\{ (x,z) \in \partial \Omega \:|\: x = 2 \wedge z \in [0,1]  \right\},\\
\Gamma_{D} = \Gamma_{D_1} \cup \Gamma_{D_2},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

The water content $\theta(\psi)$ and the conductivity tensor $K(\psi)$ can be modelled using the van Genuchten model:
$$
\theta(\psi) = \theta_{R} + \frac{\theta_{S} - \theta_{R}}{\left[ 1 + \left( -\alpha \psi \right)^n \right]^{\frac{n-1}{n} }}
$$

$$
K(\psi) = \left( \hat{\theta}(\psi) \right)^{\frac{1}{2}} K_S \left[ 1 - \left( 1 - \hat{\theta}(\psi)^{\frac{n}{n-1}} \right)^{\frac{n-1}{n}} \right]^2 \underline{\underline{\bm{I}}}
$$
where $\hat{\theta}(\psi) = \frac{\theta(\psi) - \theta_{R}}{\theta_{S} - \theta_{R}}$. Both expressions hold only if $\psi \leq 0$. If $\psi > 0$ at $\bm{x} \in \Omega$, the porous medium is fully saturated by the wetting fluid in that point and, thus, $\theta(\psi)=\theta_S$ and $K(\psi)=K_S \underline{\underline{\bm{I}}}$.

The weak formulation of the problem will read:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\left( K^{-1}(\psi) \bm{q}, \bm{v} \right)_{\Omega} - \left( \psi, \nabla \cdot \bm{v} \right)_{\Omega} = - \left( \bm{\nabla} z, \bm{v} \right)_{\Omega} - \left( \hat{\psi}, \bm{v} \cdot \nu \right)_{\Gamma_D} \\
\left(\partial_t \theta (\psi), v \right)_{\Omega} + (\nabla \cdot \bm{q}, v)_{\Omega} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

We will use the Backward Euler discretization scheme to discretize the equation in time:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\left( K^{-1}(\psi^{n+1}) \bm{q}^{n+1}, \bm{v} \right)_{\Omega} - \left( \psi^{n+1}, \nabla \cdot \bm{v} \right)_{\Omega} = - \left( \bm{\nabla} z, \bm{v} \right)_{\Omega} - \left( \hat{\psi}^{n+1}, \bm{v} \cdot \nu \right)_{\Gamma_D} \\
\left(\theta(\psi^{n+1}) , v \right)_{\Omega} + \Delta t^{n+1} (\nabla \cdot \bm{q}^{n+1}, v)_{\Omega} = \left(\theta(\psi^{n}) , v \right)_{\Omega}
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

In order to solve the problem, we will firstly have to deal with the non-linear terms. To do that, we will employ the L-scheme, obtaining:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\left( K^{-1}(\psi^{n+1}_k) \bm{q}^{n+1}_{k+1}, \bm{v} \right)_{\Omega} - \left( \psi^{n+1}_{k+1}, \nabla \cdot \bm{v} \right)_{\Omega} = - \left( \bm{\nabla} z, \bm{v} \right)_{\Omega} - \left( \hat{\psi}^{n+1}, \bm{v} \cdot \nu \right)_{\Gamma_D} \\
\left(\theta(\psi^{n+1}_k) , v \right)_{\Omega} + L\left( \psi^{n+1}_{k+1} - \psi^{n+1}_{k}, v \right)_{\Omega} + \Delta t^{n+1} (\nabla \cdot \bm{q}^{n+1}_{k+1}, v)_{\Omega} = \left(\theta(\psi^{n}) , v \right)_{\Omega}
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

The space discretization is perfomed using FEM ($RT_{0}$ elements for $\bm{q}$ and $P_0$ for $\psi$). The resulting linear system that must be solved for each step and each iteration is:
$$
\left(
\begin{array}{cc} 
M_u(\bm{\psi^{n+1}_k}) & B^T\\
-\Delta t^{n+1} B & L * M_{\psi}
\end{array}
\right)
\left(
\begin{array}{c} 
\bm{u^{n+1}_{k+1}}\\ 
\bm{\psi^{n+1}_{k+1}}\\
\end{array}
\right)
=\left(
\begin{array}{c} 
\bm{Z} + \bm{BC}^{n+1}\\ 
L * M_{\psi} \bm{\psi^{n+1}_k} + \Theta^n - \Theta^{n+1}_k
\end{array}
\right)
$$

where $[M_u(\bm{\psi^{n+1}_k})]_{ij} = \left(K^{-1}(\psi^{n+1}_k) \bm{v}_j, \bm{v}_i \right)_{\Omega}$ and $M_{\psi}$ is the $\psi$-mass matrix.

The stopping criteria for the non-linear solver is:
$$
|\bm{\psi}_{k+1}^{n+1} - \bm{\psi}_{k}^{n+1}| \leq \epsilon_{A} + \epsilon_{R} |\bm{\psi}_{k}^{n+1}|
$$

where $\epsilon_{A}$ and $\epsilon_{R}$ are the absolute and relative error tolerances.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

### Import and parameters

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
from scipy.sparse import linalg
import sympy as sp

import porepy as pp
import pygeon as pg

import time
from math import ceil, floor, log10, exp

import matplotlib.pyplot as plt

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 100

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 1/16

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-5
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'output_evolutionary_richards'

In [3]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1/n

In [4]:
# Time step
dt   = (T-0)/num_steps

### $\theta$ and $K$

In order to evaluate analytically both $\theta$ and $K$, we will make use of the python package $\textit{sympy}$, that allow us to write down a mathematical expression in symbolic form (and that can also be used to compute their derivatives).

In [5]:
# Symbolic psi
psi_var = sp.Symbol('psi', negative=True)

# Symbolic Theta
theta_expression = theta_r + (theta_s - theta_r) / (1 + (-alpha * psi_var) ** n) ** m
effective_saturation = (theta_expression - theta_r) / (theta_s - theta_r)

# Symbolic Conductivity K
hydraulic_conductivity_expression = K_s * (effective_saturation ** 0.5) * ( 1 - (1 - effective_saturation ** (1 / m)) ** m ) ** 2

In [6]:
# Theta lambda
theta_lambda = sp.lambdify(psi_var, theta_expression, 'numpy')

# Conductivity tensor lambda
conductivity_lambda = sp.lambdify(psi_var, hydraulic_conductivity_expression, 'numpy')

In [7]:
# Actual (and final) theta function
def theta(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * theta_s
    res[mask] = theta_lambda(psi[mask])

    return res

In [8]:
# Actual (and final) theta function
def conductivity(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * K_s
    res[mask] = conductivity_lambda(psi[mask])

    return res

### Domain preparation

In [9]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])

# Convert it to a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [10]:
key = "flow"

# Collection of boundary conditions
bc_value = []
bc_essential = []

# Initial pressure
initial_pressure = []

# Discretizations for q and \psi
RT0 = pg.RT0(key)
P0  = pg.PwConstants(key)

In [11]:
# Initial pressure function
def initial_pressure_func(x): 
    return 1

In [12]:
# Fake loop to extract the grid and its data (i.e. conductivity tensor)
for subdomain, data in mdg.subdomains(return_data=True):
    # Prepare the inital pressure term by interpolating initial_pressure_func into the P0 space
    initial_pressure.append(P0.interpolate(subdomain, initial_pressure_func))
        
    # Get the boundary faces ids
    boundary_faces_indexes = subdomain.get_boundary_faces()

    # Gamma_D1 and Gamma_D2 boundary faces
    gamma_d1 = np.logical_and(subdomain.face_centers[0, :] > 0-domain_tolerance, np.logical_and(subdomain.face_centers[0, :] < 1+domain_tolerance, subdomain.face_centers[1, :] > 3-domain_tolerance))
    gamma_d2 = np.logical_and(subdomain.face_centers[0, :] > 2-domain_tolerance, np.logical_and(subdomain.face_centers[1, :] > 0-domain_tolerance, subdomain.face_centers[1, :] < 1+domain_tolerance))

    gamma_d  = np.logical_or(gamma_d1, gamma_d2)

    # Gamma_N is the remaining part of the boundary    
    gamma_n  = gamma_d.copy()
    gamma_n[boundary_faces_indexes] = np.logical_not(gamma_n[boundary_faces_indexes])
    
    # Set the initial conductivity tensor in data (the actual saved tensor does not matter at this stage)
    pp.initialize_data(subdomain, data, key, {
        "second_order_tensor": pp.SecondOrderTensor(np.ones(subdomain.num_cells)),
    })
    
    # Prepare the \hat{\psi} function
    def bc_gamma_d(x, t):
        if   x[0] > 2-domain_tolerance and x[1] > 0-domain_tolerance and x[1] < 1+domain_tolerance:
            res =  1
        elif x[1] > 3-domain_tolerance and x[0] > 0-domain_tolerance and x[0] < 1+domain_tolerance:
            res = min( 3.2, 1 + 2.2 * t / dt_D )
        else:
            res = 0
        
        return res

    # Add a lambda function that generates for each time instant the (discretized) natural boundary conditions for the problem
    bc_value.append(lambda t: - RT0.assemble_nat_bc(subdomain, lambda x: bc_gamma_d(x,t), gamma_d))

    # Set the essential boundary conditions (they will be enforced before solving the system)
    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential = np.hstack((gamma_n, essential_pressure_dofs))

### Matrix assembly

In [13]:
# Psi mass matrix
M_psi = P0.assemble_mass_matrix(subdomain)

# B
B = - pg.cell_mass(mdg, P0) @ pg.div(mdg)

# Psi projection
proj_psi = P0.eval_at_cell_centers(subdomain)

# q projection
proj_q = RT0.eval_at_cell_centers(subdomain)

In [14]:
dof_psi, dof_q = B.shape
dof_psi, dof_q

(1200, 1850)

In [15]:
# Assemble initial solution
initial_solution = np.zeros(dof_q + dof_psi)
initial_solution[-dof_psi:] += np.hstack(initial_pressure)

# Final solution list. Each of its elements will be the solution at a specific instant
sol = [initial_solution]

In [16]:
# Assemble the fixed part of the right hand side (rhs)
fixed_rhs = np.zeros(dof_q + dof_psi)

In [17]:
# Helper function to project a function evaluated in the cell center to FEM (scalar)
def project_psi_to_fe(to_project):
    return to_project * subdomain.cell_volumes

### Solving stage

In [18]:
# Delete the output directory, if it exisis
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [19]:
# Helper function to export the current_sol to a file
def export_solution(saver, current_sol, num_step):
    ins = list()

    ins.append((subdomain, "cell_q", ( proj_q @ current_sol[:dof_q] ).reshape((3, -1), order="F")))
    ins.append((subdomain, "cell_h", proj_psi @ current_sol[dof_q:(dof_q+dof_psi)]))
    ins.append((subdomain, "cell_p", proj_psi @ current_sol[dof_q:(dof_q+dof_psi)] - subdomain.cell_centers[1, :]))

    saver.write_vtu(ins, time_step=num_step)

In [20]:
# Prepare the porepy exporter and export the initial solution
saver = pp.Exporter(mdg, 'sol', folder_name=output_directory)

export_solution(saver, current_sol=sol[-1], num_step=0)

In [21]:
# Time loop
for n in range(num_steps):
    current_time = (n + 1) * dt
    print('Time ' + str(round(current_time, 5)))

    # Rhs that changes with time (but not with k)
    time_rhs = fixed_rhs.copy()

    # Add the (natural) boundary conditions
    time_rhs[:dof_q] += np.hstack(list(cond(current_time) for cond in bc_value))

    # Add \Theta^n:
    # 1. Convert psi DOF to cell-wise values
    # 2. Compute theta
    # 3. Project it to P0 elements
    # 4. Multiply by psi-mass
    time_rhs[-dof_psi:] += M_psi @ project_psi_to_fe( theta( proj_psi @ sol[-1][-dof_psi:] - subdomain.cell_centers[1, :] ) ) / dt

    # Solution at the previous iteration (k=0 corresponds to the solution at the previous time step)
    prev = sol[-1]
    current = None

    debug_saver = pp.Exporter(mdg, str(current_time) + '_sol', folder_name=os.path.join(output_directory, 'debug'))
    export_solution(debug_saver, current_sol=prev, num_step=0)

    # Non-linear solver
    for k in range(K):
        # Actual rhs
        rhs = time_rhs.copy()

        # \Theta^{n+1}_k, same steps as \Theta^n
        rhs[-dof_psi:] -= M_psi @ project_psi_to_fe( theta( proj_psi @ prev[-dof_psi:] - subdomain.cell_centers[1, :] ) ) / dt
        
        # L-term
        rhs[-dof_psi:] += L * M_psi @ prev[-dof_psi:] / dt
    
        
        # Set the conductivity tensor in data (the actual saved tensor does not matter at this stage)
        cond = conductivity(proj_psi @ prev[-dof_psi:] - subdomain.cell_centers[1, :])
        pp.initialize_data(subdomain, data, key, {
            "second_order_tensor": pp.SecondOrderTensor(
                cond #2*cond, cond, None, cond
            ),
        })

        Mass_u = pg.face_mass(mdg, RT0)
        

        # Assemble the system to be solved at time n and interation k
        spp = sps.bmat(
            [[Mass_u,        B.T] ,
             [-B, L * M_psi / dt]], format="csc"
        )

        # Prepare the linear solver
        ls = pg.LinearSystem(spp, rhs)

        # Fix the essential boundary conditions
        ls.flag_ess_bc(np.hstack(bc_essential), np.zeros(dof_q + dof_psi))

        # Solve the system
        current = ls.solve()
        
        export_solution(debug_saver, current_sol=current, num_step=(k+1))

        # Check if we have reached convergence
        abs_err_psi  = np.sqrt((current[-dof_psi:] - prev[-dof_psi:]).T @ M_psi @ (current[-dof_psi:] - prev[-dof_psi:]))
        abs_err_prev = np.sqrt(prev[-dof_psi:].T @ M_psi @ prev[-dof_psi:])

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(abs_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if abs_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break
        
    print('')    
    sol.append( current )
    export_solution(saver, current_sol=sol[-1], num_step=(n+1))

saver.write_pvd([n * dt for n in range(num_steps + 1)])

Time 0.02083
Iteration #001, error L2 relative psi:    0.099373559
Iteration #002, error L2 relative psi:    0.012723616
Iteration #003, error L2 relative psi:    0.001123372
Iteration #004, error L2 relative psi:    0.000066973
Iteration #005, error L2 relative psi:    0.000009343

Time 0.04167
Iteration #001, error L2 relative psi:    0.177336693
Iteration #002, error L2 relative psi:    0.048458125
Iteration #003, error L2 relative psi:    0.010661063
Iteration #004, error L2 relative psi:    0.001259998
Iteration #005, error L2 relative psi:    0.000063268
Iteration #006, error L2 relative psi:    0.000019044

Time 0.0625
Iteration #001, error L2 relative psi:    0.266342025
Iteration #002, error L2 relative psi:    0.093789707
Iteration #003, error L2 relative psi:    0.029593627
Iteration #004, error L2 relative psi:    0.005543039
Iteration #005, error L2 relative psi:    0.001139019
Iteration #006, error L2 relative psi:    0.000447269
Iteration #007, error L2 relative psi:    

KeyboardInterrupt: 